# CZ4041 Kaggle Playground S5E9 - BPM Prediction


## 1. Data Loading and Validation


In [43]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path('.')
DATA_DIR = ROOT / 'Data'

train_path = DATA_DIR / 'train.csv'
test_path = DATA_DIR / 'test.csv'
sub_path = DATA_DIR / 'sample_submission.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
sub_df = pd.read_csv(sub_path)

print('train shape:', train_df.shape)
print('test shape:', test_df.shape)
print('sample_submission shape:', sub_df.shape)

print('\ntrain head:')
print(train_df.head())
print('\ntest head:')
print(test_df.head())
print('\nsample_submission head:')
print(sub_df.head())

print('\ntrain dtypes:')
print(train_df.dtypes)
print('\ntest dtypes:')
print(test_df.dtypes)
print('\nsample_submission dtypes:')
print(sub_df.dtypes)

print('\nmissing values (train):')
print(train_df.isna().sum())
print('\nmissing values (test):')
print(test_df.isna().sum())
print('\nmissing values (sample_submission):')
print(sub_df.isna().sum())

TARGET = 'BeatsPerMinute'
ID_COL = 'id'

assert TARGET in train_df.columns, "Expected 'BeatsPerMinute' in train.csv"
assert TARGET not in test_df.columns, "'BeatsPerMinute' must not be in test.csv"
assert ID_COL in train_df.columns, "Expected 'id' in train.csv"
assert ID_COL in test_df.columns, "Expected 'id' in test.csv"
assert list(sub_df.columns) == ['id', 'BeatsPerMinute'], "sample_submission columns must be ['id','BeatsPerMinute']"
assert len(test_df) == len(sub_df), "test row count must equal sample_submission row count"
assert np.array_equal(test_df[ID_COL].values, sub_df['id'].values), "test['id'] must match sample_submission['id'] in the same order"

feature_cols = [c for c in train_df.columns if c not in [ID_COL, TARGET]]
assert set(feature_cols) == set([c for c in test_df.columns if c != ID_COL]), "test feature columns must match train features"

print('\nfeature columns:', feature_cols)
print('feature count:', len(feature_cols))


train shape: (524164, 11)
test shape: (174722, 10)
sample_submission shape: (174722, 2)

train head:
   id  RhythmScore  AudioLoudness  VocalContent  AcousticQuality  InstrumentalScore  LivePerformanceLikelihood  \
0   0     0.603610      -7.636942      0.023500         0.000005           0.000001                   0.051385   
1   1     0.639451     -16.267598      0.071520         0.444929           0.349414                   0.170522   
2   2     0.514538     -15.953575      0.110715         0.173699           0.453814                   0.029576   
3   3     0.734463      -1.357000      0.052965         0.001651           0.159717                   0.086366   
4   4     0.532968     -13.056437      0.023500         0.068687           0.000001                   0.331345   

   MoodScore  TrackDurationMs    Energy  BeatsPerMinute  
0   0.409866      290715.6450  0.826267       147.53020  
1   0.651010      164519.5174  0.145400       136.15963  
2   0.423865      174495.5667  0.624667 

## 2. Exploratory Data Analysis


In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ARTIFACTS_DIR = Path('.') / 'artifacts'
FIG_DIR = ARTIFACTS_DIR / 'figures'
TABLE_DIR = ARTIFACTS_DIR / 'tables'

FIG_DIR.mkdir(parents=True, exist_ok=True)
TABLE_DIR.mkdir(parents=True, exist_ok=True)

# Target distribution
plt.figure(figsize=(7, 4))
plt.hist(train_df[TARGET], bins=40, color='#1f77b4', alpha=0.85)
plt.title('Target Distribution: BeatsPerMinute')
plt.xlabel('BeatsPerMinute')
plt.ylabel('Count')
plt.tight_layout()
fig_path = FIG_DIR / 'target_distribution.png'
plt.savefig(fig_path, dpi=150)
plt.close()
print('Saved:', fig_path.resolve())

# Energy histogram
plt.figure(figsize=(7, 4))
plt.hist(train_df['Energy'], bins=40, color='#ff7f0e', alpha=0.85)
plt.title('Distribution: Energy')
plt.xlabel('Energy')
plt.ylabel('Count')
plt.tight_layout()
fig_path = FIG_DIR / 'energy_distribution.png'
plt.savefig(fig_path, dpi=150)
plt.close()
print('Saved:', fig_path.resolve())

# RhythmScore histogram
plt.figure(figsize=(7, 4))
plt.hist(train_df['RhythmScore'], bins=40, color='#2ca02c', alpha=0.85)
plt.title('Distribution: RhythmScore')
plt.xlabel('RhythmScore')
plt.ylabel('Count')
plt.tight_layout()
fig_path = FIG_DIR / 'rhythmscore_distribution.png'
plt.savefig(fig_path, dpi=150)
plt.close()
print('Saved:', fig_path.resolve())

# TrackDurationMs histogram with log1p transform
plt.figure(figsize=(7, 4))
plt.hist(np.log1p(train_df['TrackDurationMs']), bins=40, color='#9467bd', alpha=0.85)
plt.title('Distribution: log1p(TrackDurationMs)')
plt.xlabel('log1p(TrackDurationMs)')
plt.ylabel('Count')
plt.tight_layout()
fig_path = FIG_DIR / 'trackduration_log1p_distribution.png'
plt.savefig(fig_path, dpi=150)
plt.close()
print('Saved:', fig_path.resolve())

# Correlation heatmap (features + target)
cols_for_corr = feature_cols + [TARGET]
corr = train_df[cols_for_corr].corr()
plt.figure(figsize=(10, 8))
plt.imshow(corr.values, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar(label='Correlation')
plt.xticks(range(len(cols_for_corr)), cols_for_corr, rotation=90)
plt.yticks(range(len(cols_for_corr)), cols_for_corr)
plt.title('Correlation Heatmap (Features + Target)')
plt.tight_layout()
fig_path = FIG_DIR / 'correlation_heatmap.png'
plt.savefig(fig_path, dpi=150)
plt.close()
print('Saved:', fig_path.resolve())

# Train vs test overlay histograms
for col, color in [('Energy', '#ff7f0e'), ('RhythmScore', '#2ca02c'), ('TrackDurationMs', '#9467bd')]:
    plt.figure(figsize=(7, 4))
    plt.hist(train_df[col], bins=40, color=color, alpha=0.6, label='train', density=True)
    plt.hist(test_df[col], bins=40, color='gray', alpha=0.4, label='test', density=True)
    plt.title(f'Train vs Test: {col}')
    plt.xlabel(col)
    plt.ylabel('Density')
    plt.legend()
    plt.tight_layout()
    fig_path = FIG_DIR / f'{col.lower()}_train_test_overlay.png'
    plt.savefig(fig_path, dpi=150)
    plt.close()
    print('Saved:', fig_path.resolve())

# Tables
train_desc = train_df.describe().T
train_desc.to_csv(TABLE_DIR / 'train_describe.csv')
print('Saved:', (TABLE_DIR / 'train_describe.csv').resolve())

test_desc = test_df.describe().T
test_desc.to_csv(TABLE_DIR / 'test_describe.csv')
print('Saved:', (TABLE_DIR / 'test_describe.csv').resolve())

corr_with_target = train_df[feature_cols].corrwith(train_df[TARGET]).sort_values(ascending=False)
corr_with_target.to_csv(TABLE_DIR / 'feature_target_correlation.csv', header=['correlation'])
print('Saved:', (TABLE_DIR / 'feature_target_correlation.csv').resolve())

# Top absolute correlations with target
abs_corr = corr_with_target.abs().sort_values(ascending=False)
print('\nTop 3 absolute correlations with target:')
print(abs_corr.head(3))

# Top KS shifts
print('\nTop 5 most shifted features:')
print(ks_df.head(5))

# KS shift check
ks_rows = []
for col in feature_cols:
    train_vals = train_df[col].values
    test_vals = test_df[col].values
    train_sorted = np.sort(train_vals)
    test_sorted = np.sort(test_vals)
    data_all = np.sort(np.concatenate([train_sorted, test_sorted]))
    train_cdf = np.searchsorted(train_sorted, data_all, side='right') / train_sorted.size
    test_cdf = np.searchsorted(test_sorted, data_all, side='right') / test_sorted.size
    ks_stat = np.max(np.abs(train_cdf - test_cdf))
    ks_rows.append({'feature': col, 'ks_stat': ks_stat})

ks_df = pd.DataFrame(ks_rows).sort_values('ks_stat', ascending=False)
ks_df.to_csv(TABLE_DIR / 'ks_shift.csv', index=False)
print('Saved:', (TABLE_DIR / 'ks_shift.csv').resolve())



## EDA Summary
- Strongest absolute correlation with target: MoodScore (|r|=0.0071).
- Strongest absolute correlation with target: TrackDurationMs (|r|=0.0066).
- Strongest absolute correlation with target: RhythmScore (|r|=0.0054).
- Largest train-vs-test shift by KS: TrackDurationMs (KS=0.0023).
- Top 5 shifted features: TrackDurationMs, MoodScore, RhythmScore, Energy, InstrumentalScore.


## 3. Baseline Model - LightGBM with 5-Fold Cross Validation

### Motivation
To establish a strong and reliable reference point, we first train a baseline model using only the raw input features. This baseline represents the best performance achievable without any feature engineering or distribution alignment.

LightGBM is chosen because gradient boosting decision trees are well suited for tabular data with nonlinear relationships and feature interactions. They also require minimal preprocessing and are robust to feature scaling.

### Method
**5-fold cross validation** with shuffling and a fixed random seed to ensure reproducibility.  
For each fold:
- the model is trained on 4 folds,
- validated on the remaining fold,
- and early stopping is used to prevent overfitting.

Out-of-fold (OOF) predictions are collected for all training samples, and the test predictions are averaged across folds.

### Evaluation
Performance is measured using **Root Mean Squared Error (RMSE)**.  
The mean and standard deviation of RMSE across the 5 folds represent the baseline performance.

All subsequent experiments are compared against this baseline to evaluate whether new features or modeling strategies provide real improvements.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import lightgbm as lgb

RESULTS_DIR = Path('.') / 'artifacts' / 'results'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

def train_cv_lgbm(X, y, X_test, n_splits=5, seed=1, use_h1=False, use_h2=False, use_quantile=True, params=None, num_boost_round=5000, early_stopping_rounds=200, return_best_iteration_mean=False):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    oof_preds = np.zeros(len(X))
    test_preds = np.zeros(len(X_test))
    fold_scores = []
    best_iterations = []

    if use_h1:
        X = add_h1_features(X)
        X_test = add_h1_features(X_test)

    numeric_features = list(X.columns)

    if params is None:
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': 0.05,
            'num_leaves': 31,
            'max_depth': -1,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 1,
            'seed': seed,
            'verbosity': -1
        }

    for fold, (tr_idx, va_idx) in enumerate(kf.split(X), start=1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        if use_h2:
            n_quantiles = min(1000, len(X_tr))
            transformers = [
                ('raw', 'passthrough', numeric_features),
                ('scaled', StandardScaler(), numeric_features)
            ]
            if use_quantile:
                transformers.append((
                    'quantile',
                    QuantileTransformer(
                        output_distribution='normal',
                        n_quantiles=n_quantiles,
                        random_state=seed
                    ),
                    numeric_features
                ))
            preprocessor = ColumnTransformer(transformers=transformers, remainder='drop')
            X_tr_proc = preprocessor.fit_transform(X_tr)
            X_va_proc = preprocessor.transform(X_va)
            X_test_proc = preprocessor.transform(X_test)
        else:
            X_tr_proc = X_tr
            X_va_proc = X_va
            X_test_proc = X_test

        lgb_train = lgb.Dataset(X_tr_proc, label=y_tr)
        lgb_valid = lgb.Dataset(X_va_proc, label=y_va, reference=lgb_train)

        model = lgb.train(
            params,
            lgb_train,
            num_boost_round=num_boost_round,
            valid_sets=[lgb_train, lgb_valid],
            valid_names=['train', 'valid'],
            callbacks=[lgb.early_stopping(early_stopping_rounds, verbose=False)]
        )

        best_iterations.append(model.best_iteration)
        va_pred = model.predict(X_va_proc, num_iteration=model.best_iteration)
        oof_preds[va_idx] = va_pred
        rmse = float(np.sqrt(mean_squared_error(y_va, va_pred)))
        fold_scores.append(rmse)
        print(f'Fold {fold} RMSE:', rmse)

        test_fold_pred = model.predict(X_test_proc, num_iteration=model.best_iteration)
        test_preds += test_fold_pred / n_splits

    mean_rmse = float(np.mean(fold_scores))
    std_rmse = float(np.std(fold_scores))
    if return_best_iteration_mean:
        best_iteration_mean = float(np.mean(best_iterations))
        return oof_preds, test_preds, fold_scores, mean_rmse, std_rmse, best_iteration_mean
    return oof_preds, test_preds, fold_scores, mean_rmse, std_rmse

X = train_df[feature_cols]
y = train_df[TARGET]
X_test = test_df[feature_cols]

oof_preds, test_preds, fold_scores, mean_rmse, std_rmse = train_cv_lgbm(X, y, X_test, n_splits=5, 1, use_h1=False)

baseline_mean_rmse = mean_rmse
baseline_std_rmse = std_rmse
print('\nMean RMSE:', mean_rmse)
print('Std RMSE:', std_rmse)

baseline_folds = pd.DataFrame({'fold': list(range(1, len(fold_scores) + 1)), 'rmse': fold_scores})
baseline_folds_path = RESULTS_DIR / 'baseline_folds.csv'
baseline_folds.to_csv(baseline_folds_path, index=False)
print('Saved:', baseline_folds_path.resolve())

baseline_oof = pd.DataFrame({ID_COL: train_df[ID_COL], 'oof_pred': oof_preds})
baseline_oof_path = RESULTS_DIR / 'baseline_oof.csv'
baseline_oof.to_csv(baseline_oof_path, index=False)
print('Saved:', baseline_oof_path.resolve())

submission_cols = sub_df.columns.tolist()
sub = pd.DataFrame({submission_cols[0]: test_df[ID_COL], submission_cols[1]: test_preds})
submission_path = RESULTS_DIR / 'submission_baseline.csv'
sub.to_csv(submission_path, index=False)
print('Saved:', submission_path.resolve())
print('\nSubmission head:')
print(sub.head())


This baseline represents the performance using raw features only. All further improvements are compared against this score.


## 4. Feature Engineering - Hypothesis 1 (Interaction Geometry)
# Motivation

Beats Per Minute (BPM) is not determined by any single audio characteristic in isolation. Instead, tempo emerges from the interaction of multiple perceptual factors, such as energy, rhythm, loudness, and duration.

For example, fast songs typically exhibit:

- high rhythmic structure,

- high perceived energy,

- higher loudness,

- and shorter durations.

However, these relationships are nonlinear and coupled. A track with high energy but low rhythmic structure may not feel fast, while a highly rhythmic track with moderate energy may still feel fast. Tree-based models can learn some of these interactions, but explicitly encoding them can help the model generalize more reliably, especially under slight distribution shifts.

Therefore, Hypothesis 1 proposes that explicit interaction and nonlinear features will better represent the latent structure that governs perceived tempo.

# Feature Design

To capture this geometry, we introduce three groups of features:

1) Nonlinear transforms

These allow the model to represent curvature in the relationship between features and BPM.

- log(TrackDurationMs + 1)

- Energy²

- RhythmScore²

- AudioLoudness²

These terms capture diminishing or accelerating effects that a linear feature cannot represent.

2) Pairwise interactions

These encode how two perceptual dimensions jointly influence tempo.

- Energy × RhythmScore

- Energy × AudioLoudness

- RhythmScore × AudioLoudness

These reflect that BPM perception depends on combined intensity and structure, not just independent effects.

3) Scale-normalized ratios

These control for track length, which strongly influences perceived tempo.

- Energy / log(TrackDurationMs + 1)

- RhythmScore / log(TrackDurationMs + 1)

Short, energetic songs tend to feel faster than long, energetic songs. These ratios normalize energy and rhythm by duration.

4) Optional composite cues

- Energy × MoodScore

- (Energy + RhythmScore) / 2

These represent higher-level cues of drive and rhythmic emphasis.

#Experimental Goal

The objective of H1 is to test whether explicitly encoding nonlinear interactions improves predictive performance over using raw features alone.

We compare:

| Model    | Features                   | RMSE  |
| -------- | -------------------------- | ----- |
| Baseline | Raw features               | X     |
| H1       | Raw + interaction features | X − Δ |


If Δ > 0, it supports the hypothesis that BPM emerges from interaction geometry rather than linear contributions.

This approach is:

- interpretable (each feature has a perceptual meaning),

- experimentally testable (via ablation),

- and grounded in how humans perceive tempo.

It moves the model from “fitting numbers” to modeling the structure of perception, which strengthens both performance and scientific credibility.

In [ ]:
USE_H1 = True

def add_h1_features(df):
    df = df.copy()
    eps = 1e-6
    log_duration = np.log1p(df['TrackDurationMs'])
    df['log1p_TrackDurationMs'] = log_duration
    df['Energy_sq'] = df['Energy'] ** 2
    df['RhythmScore_sq'] = df['RhythmScore'] ** 2
    df['AudioLoudness_sq'] = df['AudioLoudness'] ** 2
    df['Energy_x_RhythmScore'] = df['Energy'] * df['RhythmScore']
    df['Energy_x_AudioLoudness'] = df['Energy'] * df['AudioLoudness']
    df['RhythmScore_x_AudioLoudness'] = df['RhythmScore'] * df['AudioLoudness']
    denom = log_duration + eps
    df['Energy_div_logDuration'] = df['Energy'] / denom
    df['Rhythm_div_logDuration'] = df['RhythmScore'] / denom
    df['Energy_x_MoodScore'] = df['Energy'] * df['MoodScore']
    df['Energy_Rhythm_mean'] = (df['Energy'] + df['RhythmScore']) / 2.0
    return df

X = train_df[feature_cols]
y = train_df[TARGET]
X_test = test_df[feature_cols]

oof_h1, test_h1, h1_fold_scores, h1_mean_rmse, h1_std_rmse = train_cv_lgbm(
    X, y, X_test, n_splits=5, seed=1, use_h1=USE_H1
)

print('\nBaseline mean RMSE:', baseline_mean_rmse)
print('H1 mean RMSE:', h1_mean_rmse)
print('Delta (baseline - H1):', baseline_mean_rmse - h1_mean_rmse)

h1_folds = pd.DataFrame({'fold': list(range(1, len(h1_fold_scores) + 1)), 'rmse': h1_fold_scores})
h1_folds_path = RESULTS_DIR / 'h1_folds.csv'
h1_folds.to_csv(h1_folds_path, index=False)
print('Saved:', h1_folds_path.resolve())

h1_oof = pd.DataFrame({ID_COL: train_df[ID_COL], 'oof_pred': oof_h1})
h1_oof_path = RESULTS_DIR / 'h1_oof.csv'
h1_oof.to_csv(h1_oof_path, index=False)
print('Saved:', h1_oof_path.resolve())


### Hypothesis 1 Results Analysis
H1 does not improve performance (Δ = −0.0005 RMSE). This does not support the hypothesis that explicitly modeling nonlinear interactions between energy, rhythm, loudness, and duration improves BPM prediction under this setting.

## 5. Feature Engineering - Hypothesis 2 (Distribution Alignment)

### Motivation
The dataset used in this competition was generated by a deep learning model rather than collected from real audio recordings. As a result, the training and test sets are not guaranteed to follow exactly the same feature distributions, even if they share similar ranges and semantics.

In such synthetic settings, models may overfit to the **absolute scale and shape** of the training distribution, which can reduce generalization when the test data is slightly shifted. However, the **relative ordering and rank structure** of features often remains more stable than raw values.

Therefore, Hypothesis 2 proposes that **aligning feature distributions** improves generalization and reduces RMSE.

### Feature Design
For each numeric feature, we construct distribution-aligned representations using:

1) **Standardized features (z-score)**  
Each feature is centered and scaled to unit variance using statistics computed from the training fold only.

2) **Quantile-transformed features**  
Each feature is mapped to a Gaussian-like distribution using a quantile transformation fitted on the training fold. This preserves relative ordering while reducing skewness and scale differences.

The final H2 representation concatenates:
- raw features  
- standardized features  
- quantile-transformed features  

This provides the model with multiple views of the same information: absolute scale, relative scale, and normalized rank structure.

All transformations are applied using a cross-validation pipeline to avoid data leakage.

### Experimental Goal
We evaluate H2 by comparing:

| Model    | Features                           | RMSE   |
|----------|------------------------------------|--------|
| Baseline | Raw features                        | X      |
| H2       | Raw + distribution-aligned features | X - Δ  |

If Δ > 0, it supports the hypothesis that distribution alignment improves generalization under synthetic train-test shift.

### Why this is principled
This approach is:
- statistically grounded in distribution normalization,
- robust to scale and shape differences between datasets,
- and well suited to synthetic or model-generated data.

H2 focuses not on increasing model capacity, but on improving **how the data is represented**, which can lead to more stable and transferable predictions.



In [ ]:
USE_H2 = True
USE_QUANTILE = True

X = train_df[feature_cols]
y = train_df[TARGET]
X_test = test_df[feature_cols]

# Exp2: H2 only
oof_h2, test_h2, h2_fold_scores, h2_mean_rmse, h2_std_rmse = train_cv_lgbm(
    X, y, X_test, n_splits=5, seed=1, use_h1=False, use_h2=USE_H2, use_quantile=USE_QUANTILE
)

print('\nH2 mean RMSE:', h2_mean_rmse)
print('H2 std RMSE:', h2_std_rmse)
print('Delta vs baseline:', baseline_mean_rmse - h2_mean_rmse)

h2_folds = pd.DataFrame({'fold': list(range(1, len(h2_fold_scores) + 1)), 'rmse': h2_fold_scores})
h2_folds_path = RESULTS_DIR / 'h2_only_folds.csv'
h2_folds.to_csv(h2_folds_path, index=False)
print('Saved:', h2_folds_path.resolve())

h2_oof = pd.DataFrame({ID_COL: train_df[ID_COL], 'oof_pred': oof_h2})
h2_oof_path = RESULTS_DIR / 'h2_only_oof.csv'
h2_oof.to_csv(h2_oof_path, index=False)
print('Saved:', h2_oof_path.resolve())

# Exp3: H1 + H2
oof_h1_h2, test_h1_h2, h1_h2_fold_scores, h1_h2_mean_rmse, h1_h2_std_rmse = train_cv_lgbm(
    X, y, X_test, n_splits=5, seed=1, use_h1=True, use_h2=USE_H2, use_quantile=USE_QUANTILE
)

print('\nH1+H2 mean RMSE:', h1_h2_mean_rmse)
print('H1+H2 std RMSE:', h1_h2_std_rmse)
print('Delta vs baseline:', baseline_mean_rmse - h1_h2_mean_rmse)

h1_h2_folds = pd.DataFrame({'fold': list(range(1, len(h1_h2_fold_scores) + 1)), 'rmse': h1_h2_fold_scores})
h1_h2_folds_path = RESULTS_DIR / 'h1_h2_folds.csv'
h1_h2_folds.to_csv(h1_h2_folds_path, index=False)
print('Saved:', h1_h2_folds_path.resolve())

h1_h2_oof = pd.DataFrame({ID_COL: train_df[ID_COL], 'oof_pred': oof_h1_h2})
h1_h2_oof_path = RESULTS_DIR / 'h1_h2_oof.csv'
h1_h2_oof.to_csv(h1_h2_oof_path, index=False)
print('Saved:', h1_h2_oof_path.resolve())

experiments = pd.DataFrame([
    {
        'experiment_name': 'baseline',
        'use_h1': False,
        'use_h2': False,
        'use_quantile': False,
        'mean_rmse': baseline_mean_rmse,
        'std_rmse': baseline_std_rmse
    },
    {
        'experiment_name': 'h1',
        'use_h1': True,
        'use_h2': False,
        'use_quantile': False,
        'mean_rmse': h1_mean_rmse,
        'std_rmse': h1_std_rmse
    },
    {
        'experiment_name': 'h2_only',
        'use_h1': False,
        'use_h2': True,
        'use_quantile': USE_QUANTILE,
        'mean_rmse': h2_mean_rmse,
        'std_rmse': h2_std_rmse
    },
    {
        'experiment_name': 'h1_h2',
        'use_h1': True,
        'use_h2': True,
        'use_quantile': USE_QUANTILE,
        'mean_rmse': h1_h2_mean_rmse,
        'std_rmse': h1_h2_std_rmse
    }
])

experiments_path = RESULTS_DIR / 'experiments.csv'
experiments.to_csv(experiments_path, index=False)
print('Saved:', experiments_path.resolve())

h2_delta = baseline_mean_rmse - h2_mean_rmse
h1_h2_delta = baseline_mean_rmse - h1_h2_mean_rmse
print('\nInterpretation (H2):')
print('H2 improves by' if h2_delta > 0 else 'H2 does not improve, delta', h2_delta)
print('H1+H2 improves by' if h1_h2_delta > 0 else 'H1+H2 does not improve, delta', h1_h2_delta)


## 6. Hyperparameter Tuning (LightGBM)


In [ ]:
import time

def random_search_lgbm(X, y, n_trials=25, seed=1):
    rng = np.random.default_rng(seed)
    results = []
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)

    def sample_params():
        lr = float(np.exp(rng.uniform(np.log(0.01), np.log(0.2))))
        num_leaves = int(rng.integers(16, 513))
        max_depth = int(rng.integers(3, 13))
        if rng.random() < 0.2:
            max_depth = -1
        min_child_samples = int(rng.integers(5, 201))
        subsample = float(rng.uniform(0.6, 1.0))
        colsample = float(rng.uniform(0.6, 1.0))
        reg_alpha = float(np.exp(rng.uniform(np.log(1e-8), np.log(10.0))))
        reg_lambda = float(np.exp(rng.uniform(np.log(1e-8), np.log(10.0))))
        min_split_gain = float(rng.uniform(0.0, 0.5))
        return {
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': lr,
            'num_leaves': num_leaves,
            'max_depth': max_depth,
            'min_child_samples': min_child_samples,
            'subsample': subsample,
            'colsample_bytree': colsample,
            'reg_alpha': reg_alpha,
            'reg_lambda': reg_lambda,
            'min_split_gain': min_split_gain,
            'seed': seed,
            'verbosity': -1
        }

    for trial in range(1, n_trials + 1):
        params = sample_params()
        start = time.time()
        fold_scores = []
        best_iters = []
        for tr_idx, va_idx in kf.split(X):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

            lgb_train = lgb.Dataset(X_tr, label=y_tr)
            lgb_valid = lgb.Dataset(X_va, label=y_va, reference=lgb_train)

            model = lgb.train(
                params,
                lgb_train,
                num_boost_round=10000,
                valid_sets=[lgb_train, lgb_valid],
                valid_names=['train', 'valid'],
                callbacks=[lgb.early_stopping(200, verbose=False)]
            )

            va_pred = model.predict(X_va, num_iteration=model.best_iteration)
            rmse = float(np.sqrt(mean_squared_error(y_va, va_pred)))
            fold_scores.append(rmse)
            best_iters.append(model.best_iteration)

        mean_rmse = float(np.mean(fold_scores))
        std_rmse = float(np.std(fold_scores))
        runtime_sec = float(time.time() - start)
        best_iter_mean = float(np.mean(best_iters))

        results.append({
            'trial_id': trial,
            'params': params,
            'mean_rmse': mean_rmse,
            'std_rmse': std_rmse,
            'best_iteration_mean': best_iter_mean,
            'runtime_sec': runtime_sec
        })
        print(f'Trial {trial}: mean RMSE={mean_rmse:.6f}, std RMSE={std_rmse:.6f}')

    results_df = pd.DataFrame(results).sort_values('mean_rmse', ascending=True)
    best_params = results_df.iloc[0]['params']
    return results_df, best_params

X = train_df[feature_cols]
y = train_df[TARGET]

tuning_results, best_params = random_search_lgbm(X, y, n_trials=25, seed=1)
tuning_path = RESULTS_DIR / 'lgbm_tuning.csv'
tuning_results.to_csv(tuning_path, index=False)
print('Saved:', tuning_path.resolve())
print('\nTop 5 trials:')
print(tuning_results.head(5))
print('\nBest params:')
print(best_params)

oof_best, test_best, best_fold_scores, best_mean_rmse, best_std_rmse, best_iter_mean = train_cv_lgbm(
    X, y, test_df[feature_cols], n_splits=5, seed=1, use_h1=False, use_h2=False, use_quantile=False,
    params=best_params, num_boost_round=10000, early_stopping_rounds=200, return_best_iteration_mean=True
)

best_folds = pd.DataFrame({'fold': list(range(1, len(best_fold_scores) + 1)), 'rmse': best_fold_scores})
best_folds_path = RESULTS_DIR / 'lgbm_best_folds.csv'
best_folds.to_csv(best_folds_path, index=False)
print('Saved:', best_folds_path.resolve())

best_oof = pd.DataFrame({ID_COL: train_df[ID_COL], 'oof_pred': oof_best})
best_oof_path = RESULTS_DIR / 'lgbm_best_oof.csv'
best_oof.to_csv(best_oof_path, index=False)
print('Saved:', best_oof_path.resolve())

submission_cols = sub_df.columns.tolist()
sub_best = pd.DataFrame({submission_cols[0]: test_df[ID_COL], submission_cols[1]: test_best})
sub_best_path = RESULTS_DIR / 'submission_lgbm_best.csv'
sub_best.to_csv(sub_best_path, index=False)
print('Saved:', sub_best_path.resolve())
print('\nBest model RMSE:', best_mean_rmse)
print('Best model std RMSE:', best_std_rmse)


## 7. Ensembling - Blend / Stacking


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

try:
    from catboost import CatBoostRegressor
    import lightgbm as lgb
except Exception:
    print('Missing CatBoost or LightGBM. Install with: pip install catboost lightgbm')
    raise SystemExit(0)

TARGET = 'BeatsPerMinute'
ID_COL = 'id'
SEED = 1
N_SPLITS = 5

DATA_DIR = Path('./Data')
RESULTS_DIR = Path('./artifacts/results')
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

if 'train_df' not in globals() or 'test_df' not in globals() or 'sub_df' not in globals():
    train_df = pd.read_csv(DATA_DIR / 'train.csv')
    test_df = pd.read_csv(DATA_DIR / 'test.csv')
    sub_df = pd.read_csv(DATA_DIR / 'sample_submission.csv')

feature_cols = [c for c in train_df.columns if c not in [ID_COL, TARGET]]
X = train_df[feature_cols]
y = train_df[TARGET]
X_test = test_df[feature_cols]

np.random.seed(SEED)
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# LightGBM CV
lgb_oof = np.zeros(len(X))
lgb_test = np.zeros(len(X_test))
lgb_fold_scores = []

params_lgb = dict(
    n_estimators=20000,
    learning_rate=0.03,
    num_leaves=256,
    max_depth=-1,
    min_child_samples=40,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.0,
    reg_lambda=1.0,
    random_state=SEED,
    n_jobs=-1
)

for fold, (tr_idx, va_idx) in enumerate(kf.split(X), start=1):
    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    model = lgb.LGBMRegressor(**params_lgb)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(300, verbose=False)]
    )

    best_iter = getattr(model, 'best_iteration_', None)
    if best_iter is None:
        va_pred = model.predict(X_va)
        test_fold = model.predict(X_test)
    else:
        va_pred = model.predict(X_va, num_iteration=best_iter)
        test_fold = model.predict(X_test, num_iteration=best_iter)

    rmse = float(np.sqrt(mean_squared_error(y_va, va_pred)))
    lgb_fold_scores.append(rmse)
    lgb_oof[va_idx] = va_pred
    lgb_test += test_fold / N_SPLITS
    print(f'LGB Fold {fold} RMSE:', rmse)

lgb_mean = float(np.mean(lgb_fold_scores))
lgb_std = float(np.std(lgb_fold_scores))
print('LGB Mean RMSE:', lgb_mean)
print('LGB Std RMSE:', lgb_std)

lgb_oof_df = pd.DataFrame({ID_COL: train_df[ID_COL], 'oof_pred': lgb_oof})
lgb_oof_path = RESULTS_DIR / 'lgb_oof_cbblend.csv'
lgb_oof_df.to_csv(lgb_oof_path, index=False)
print('Saved:', lgb_oof_path.resolve())

lgb_test_df = pd.DataFrame({ID_COL: test_df[ID_COL], 'pred': lgb_test})
lgb_test_path = RESULTS_DIR / 'lgb_test_cbblend.csv'
lgb_test_df.to_csv(lgb_test_path, index=False)
print('Saved:', lgb_test_path.resolve())

lgb_folds_df = pd.DataFrame({'fold': list(range(1, len(lgb_fold_scores) + 1)), 'rmse': lgb_fold_scores})
lgb_folds_path = RESULTS_DIR / 'lgb_folds_cbblend.csv'
lgb_folds_df.to_csv(lgb_folds_path, index=False)
print('Saved:', lgb_folds_path.resolve())

# CatBoost CV
cb_oof = np.zeros(len(X))
cb_test = np.zeros(len(X_test))
cb_fold_scores = []

params_cb = dict(
    loss_function='RMSE',
    iterations=20000,
    learning_rate=0.03,
    depth=10,
    l2_leaf_reg=5.0,
    random_seed=SEED,
    eval_metric='RMSE',
    od_type='Iter',
    od_wait=300,
    verbose=False
)

for fold, (tr_idx, va_idx) in enumerate(kf.split(X), start=1):
    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    model = CatBoostRegressor(**params_cb)
    model.fit(X_tr, y_tr, eval_set=(X_va, y_va), use_best_model=True)

    va_pred = model.predict(X_va)
    test_fold = model.predict(X_test)

    rmse = float(np.sqrt(mean_squared_error(y_va, va_pred)))
    cb_fold_scores.append(rmse)
    cb_oof[va_idx] = va_pred
    cb_test += test_fold / N_SPLITS
    print(f'CB Fold {fold} RMSE:', rmse)

cb_mean = float(np.mean(cb_fold_scores))
cb_std = float(np.std(cb_fold_scores))
print('CB Mean RMSE:', cb_mean)
print('CB Std RMSE:', cb_std)

cb_oof_df = pd.DataFrame({ID_COL: train_df[ID_COL], 'oof_pred': cb_oof})
cb_oof_path = RESULTS_DIR / 'cb_oof.csv'
cb_oof_df.to_csv(cb_oof_path, index=False)
print('Saved:', cb_oof_path.resolve())

cb_test_df = pd.DataFrame({ID_COL: test_df[ID_COL], 'pred': cb_test})
cb_test_path = RESULTS_DIR / 'cb_test_preds.csv'
cb_test_df.to_csv(cb_test_path, index=False)
print('Saved:', cb_test_path.resolve())

cb_folds_df = pd.DataFrame({'fold': list(range(1, len(cb_fold_scores) + 1)), 'rmse': cb_fold_scores})
cb_folds_path = RESULTS_DIR / 'cb_folds.csv'
cb_folds_df.to_csv(cb_folds_path, index=False)
print('Saved:', cb_folds_path.resolve())

# OOF blending
weights = np.linspace(0.0, 1.0, 51)
blend_rows = []
for w in weights:
    blended_oof = w * lgb_oof + (1.0 - w) * cb_oof
    rmse = float(np.sqrt(mean_squared_error(y, blended_oof)))
    blend_rows.append({'weight_lgbm': w, 'rmse': rmse})

blend_df = pd.DataFrame(blend_rows).sort_values('rmse', ascending=True)
blend_path = RESULTS_DIR / 'blend_weight_search_lgb_cb.csv'
blend_df.to_csv(blend_path, index=False)
print('Saved:', blend_path.resolve())

best_w = float(blend_df.iloc[0]['weight_lgbm'])
best_blend_rmse = float(blend_df.iloc[0]['rmse'])
print('Best blend weight (LGBM):', best_w)
print('Best blended RMSE:', best_blend_rmse)

blend_test = best_w * lgb_test + (1.0 - best_w) * cb_test
submission = sub_df.copy()
submission['BeatsPerMinute'] = blend_test
sub_blend_path = RESULTS_DIR / 'submission_blend_lgb_cb.csv'
submission.to_csv(sub_blend_path, index=False)
print('Saved:', sub_blend_path.resolve())
print('\nSubmission head:')
print(submission.head())


## 8. Seed Averaged LightGBM


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

TARGET = 'BeatsPerMinute'
ID_COL = 'id'
SEED_LIST = [1]
N_SPLITS = 5

DATA_DIR = Path('./Data')
RESULTS_DIR = Path('./artifacts/results')
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

if 'train_df' not in globals() or 'test_df' not in globals() or 'sub_df' not in globals():
    train_df = pd.read_csv(DATA_DIR / 'train.csv')
    test_df = pd.read_csv(DATA_DIR / 'test.csv')
    sub_df = pd.read_csv(DATA_DIR / 'sample_submission.csv')

feature_cols = [c for c in train_df.columns if c not in [ID_COL, TARGET]]
X = train_df[feature_cols]
y = train_df[TARGET]
X_test = test_df[feature_cols]

tuning_path = RESULTS_DIR / 'lgbm_tuning.csv'
params = None
if tuning_path.exists():
    tuning_df = pd.read_csv(tuning_path)
    if 'params' in tuning_df.columns:
        best_params_raw = tuning_df.loc[tuning_df['mean_rmse'].idxmin(), 'params']
        try:
            best_params = eval(best_params_raw)
            if isinstance(best_params, dict):
                params = best_params
        except Exception:
            params = None

if params is None:
    params = dict(
        objective='regression',
        metric='rmse',
        learning_rate=0.03,
        num_leaves=256,
        max_depth=-1,
        min_child_samples=40,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.0,
        reg_lambda=1.0
    )

params.pop('random_state', None)
params.pop('seed', None)

kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
oof_list = []
test_list = []
seed_scores = []

for seed in SEED_LIST:
    oof_seed = np.zeros(len(X))
    test_seed = np.zeros(len(X_test))

    for tr_idx, va_idx in kf.split(X):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        model = lgb.LGBMRegressor(
            **params,
            n_estimators=20000,
            random_state=seed,
            n_jobs=-1
        )
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric='rmse',
            callbacks=[lgb.early_stopping(300, verbose=False)]
        )

        best_iter = getattr(model, 'best_iteration_', None)
        if best_iter is None:
            va_pred = model.predict(X_va)
            test_fold = model.predict(X_test)
        else:
            va_pred = model.predict(X_va, num_iteration=best_iter)
            test_fold = model.predict(X_test, num_iteration=best_iter)

        oof_seed[va_idx] = va_pred
        test_seed += test_fold / N_SPLITS

    rmse_seed = float(np.sqrt(mean_squared_error(y, oof_seed)))
    oof_list.append(oof_seed)
    test_list.append(test_seed)
    seed_scores.append({'seed': seed, 'rmse': rmse_seed})
    print(f'Seed {seed} RMSE:', rmse_seed)

oof_avg = np.mean(np.vstack(oof_list), axis=0)
test_avg = np.mean(np.vstack(test_list), axis=0)
avg_rmse = float(np.sqrt(mean_squared_error(y, oof_avg)))
print('Averaged OOF RMSE:', avg_rmse)

oof_df = pd.DataFrame({ID_COL: train_df[ID_COL], 'oof_pred': oof_avg})
oof_path = RESULTS_DIR / 'lgbm_tuned_seedavg_oof.csv'
oof_df.to_csv(oof_path, index=False)
print('Saved:', oof_path.resolve())

submission = sub_df.copy()
submission['BeatsPerMinute'] = test_avg
submission_path = RESULTS_DIR / 'submission_lgbm_tuned_seedavg.csv'
submission.to_csv(submission_path, index=False)
print('Saved:', submission_path.resolve())
print('\nSubmission head:')
print(submission.head())

seed_scores_df = pd.DataFrame(seed_scores)
seed_scores_path = RESULTS_DIR / 'lgbm_tuned_seedavg_seed_scores.csv'
seed_scores_df.to_csv(seed_scores_path, index=False)
print('Saved:', seed_scores_path.resolve())


## 8.5 Improved Seed Averaging (LightGBM)


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

TARGET = 'BeatsPerMinute'
ID_COL = 'id'
SEED_LIST = [1,2,3]
N_SPLITS = 5

DATA_DIR = Path('./Data')
RESULTS_DIR = Path('./artifacts/results')
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

if 'train_df' not in globals() or 'test_df' not in globals() or 'sub_df' not in globals():
    train_df = pd.read_csv(DATA_DIR / 'train.csv')
    test_df = pd.read_csv(DATA_DIR / 'test.csv')
    sub_df = pd.read_csv(DATA_DIR / 'sample_submission.csv')

feature_cols = [c for c in train_df.columns if c not in [ID_COL, TARGET]]
X = train_df[feature_cols]
y = train_df[TARGET]
X_test = test_df[feature_cols]

tuning_path = RESULTS_DIR / 'lgbm_tuning.csv'
best_params = None
if tuning_path.exists():
    tuning_df = pd.read_csv(tuning_path)
    if 'params' in tuning_df.columns:
        best_params_raw = tuning_df.loc[tuning_df['mean_rmse'].idxmin(), 'params']
        try:
            parsed = eval(best_params_raw)
            if isinstance(parsed, dict):
                best_params = parsed
        except Exception:
            best_params = None

if best_params is None:
    best_params = dict(
        objective='regression',
        metric='rmse',
        learning_rate=0.03,
        num_leaves=256,
        max_depth=-1,
        min_child_samples=40,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.0,
        reg_lambda=1.0
    )

best_params.pop('random_state', None)
best_params.pop('seed', None)

# Two param variants to average (stability + small diversity)
param_grid = [
    dict(best_params),
    dict(best_params, learning_rate=0.04, num_leaves=192, min_child_samples=30),
    dict(best_params, learning_rate=0.02, num_leaves=320, min_child_samples=50)
]

kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
oof_runs = []
test_runs = []
run_scores = []

for p_idx, params in enumerate(param_grid, start=1):
    for seed in SEED_LIST:
        oof = np.zeros(len(X))
        test_pred = np.zeros(len(X_test))

        for tr_idx, va_idx in kf.split(X):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

            model = lgb.LGBMRegressor(
                **params,
                n_estimators=30000,
                random_state=seed,
                n_jobs=-1
            )
            model.fit(
                X_tr, y_tr,
                eval_set=[(X_va, y_va)],
                eval_metric='rmse',
                callbacks=[lgb.early_stopping(500, verbose=False)]
            )

            best_iter = getattr(model, 'best_iteration_', None)
            if best_iter is None:
                va_pred = model.predict(X_va)
                test_fold = model.predict(X_test)
            else:
                va_pred = model.predict(X_va, num_iteration=best_iter)
                test_fold = model.predict(X_test, num_iteration=best_iter)

            oof[va_idx] = va_pred
            test_pred += test_fold / N_SPLITS

        rmse = float(np.sqrt(mean_squared_error(y, oof)))
        oof_runs.append(oof)
        test_runs.append(test_pred)
        run_scores.append({'param_set': p_idx, 'seed': seed, 'rmse': rmse})
        print(f'ParamSet {p_idx} Seed {seed} RMSE:', rmse)

oof_avg = np.mean(np.vstack(oof_runs), axis=0)
test_avg = np.mean(np.vstack(test_runs), axis=0)
avg_rmse = float(np.sqrt(mean_squared_error(y, oof_avg)))
print('Averaged OOF RMSE:', avg_rmse)

oof_df = pd.DataFrame({ID_COL: train_df[ID_COL], 'oof_pred': oof_avg})
oof_path = RESULTS_DIR / 'lgbm_tuned_seedavg_v3_oof.csv'
oof_df.to_csv(oof_path, index=False)
print('Saved:', oof_path.resolve())

run_scores_df = pd.DataFrame(run_scores)
run_scores_path = RESULTS_DIR / 'lgbm_tuned_seedavg_v3_run_scores.csv'
run_scores_df.to_csv(run_scores_path, index=False)
print('Saved:', run_scores_path.resolve())

submission = sub_df.copy()
submission['BeatsPerMinute'] = test_avg
submission_path = RESULTS_DIR / 'submission_lgbm_tuned_seedavg_v3.csv'
submission.to_csv(submission_path, index=False)
print('Saved:', submission_path.resolve())
print('\nSubmission head:')
print(submission.head())


## 9. True Stacking (Meta-Model on OOF Predictions)


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_squared_error

TARGET = 'BeatsPerMinute'
ID_COL = 'id'
SEED = 1

DATA_DIR = Path('./Data')
RESULTS_DIR = Path('./artifacts/results')
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

if 'train_df' not in globals() or 'test_df' not in globals() or 'sub_df' not in globals():
    train_df = pd.read_csv(DATA_DIR / 'train.csv')
    test_df = pd.read_csv(DATA_DIR / 'test.csv')
    sub_df = pd.read_csv(DATA_DIR / 'sample_submission.csv')

y = train_df[TARGET].values

def _pick_pred_col(df, candidates):
    for col in candidates:
        if col in df.columns:
            return col
    num_cols = [c for c in df.columns if c != ID_COL]
    if len(num_cols) == 1:
        return num_cols[0]
    return None

def load_pred_pair(oof_path, test_path, oof_col_guess=None, test_col_guess=None):
    if not oof_path.exists() or not test_path.exists():
        return None, None
    oof_df = pd.read_csv(oof_path)
    test_df_local = pd.read_csv(test_path)

    oof_col = _pick_pred_col(oof_df, [oof_col_guess, 'oof_pred', 'BeatsPerMinute', 'pred'])
    test_col = _pick_pred_col(test_df_local, [test_col_guess, 'BeatsPerMinute', 'pred'])
    if oof_col is None or test_col is None:
        return None, None

    oof_merged = train_df[[ID_COL]].merge(oof_df[[ID_COL, oof_col]], on=ID_COL, how='left')
    test_merged = test_df[[ID_COL]].merge(test_df_local[[ID_COL, test_col]], on=ID_COL, how='left')
    if oof_merged[oof_col].isna().any() or test_merged[test_col].isna().any():
        return None, None

    return oof_merged[oof_col].values, test_merged[test_col].values

models = []
oof_list = []
test_list = []

# LightGBM seed-avg (preferred)
oof, test = load_pred_pair(
    RESULTS_DIR / 'lgbm_tuned_seedavg_oof.csv',
    RESULTS_DIR / 'submission_lgbm_tuned_seedavg.csv',
    oof_col_guess='oof_pred',
    test_col_guess='BeatsPerMinute'
)
if oof is None or test is None:
    oof, test = load_pred_pair(
        RESULTS_DIR / 'lgbm_best_oof.csv',
        RESULTS_DIR / 'submission_lgbm_best.csv',
        oof_col_guess='oof_pred',
        test_col_guess='BeatsPerMinute'
    )
    if oof is None or test is None:
        oof, test = load_pred_pair(
            RESULTS_DIR / 'baseline_oof.csv',
            RESULTS_DIR / 'submission_baseline.csv',
            oof_col_guess='oof_pred',
            test_col_guess='BeatsPerMinute'
        )
if oof is not None and test is not None:
    models.append('lgbm')
    oof_list.append(oof)
    test_list.append(test)

# CatBoost
oof, test = load_pred_pair(
    RESULTS_DIR / 'cb_oof.csv',
    RESULTS_DIR / 'cb_test_preds.csv',
    oof_col_guess='oof_pred',
    test_col_guess='pred'
)
if oof is not None and test is not None:
    models.append('catboost')
    oof_list.append(oof)
    test_list.append(test)

# XGBoost
oof, test = load_pred_pair(
    RESULTS_DIR / 'xgb_oof.csv',
    RESULTS_DIR / 'xgb_test_preds.csv',
    oof_col_guess='oof_pred',
    test_col_guess='pred'
)
if oof is not None and test is not None:
    models.append('xgboost')
    oof_list.append(oof)
    test_list.append(test)

if len(models) < 2:
    print('Need at least 2 base models for stacking. Check saved OOF/test predictions.')
    raise SystemExit(0)

X_meta_train = np.column_stack(oof_list)
X_meta_test = np.column_stack(test_list)
print('Base models used:', models)
print('X_meta_train shape:', X_meta_train.shape)

grid = np.linspace(0.0, 1.0, 51)
weights = []
if len(models) == 2:
    for w in grid:
        weights.append([w, 1.0 - w])
elif len(models) == 3:
    for w1 in grid:
        for w2 in grid:
            w3 = 1.0 - w1 - w2
            if w3 < 0:
                continue
            weights.append([w1, w2, w3])
else:
    print('This constrained search supports 2 or 3 models only.')
    raise SystemExit(0)

weights = np.array(weights)
blend_rows = []
for w in weights:
    blended_oof = X_meta_train @ w
    rmse = float(np.sqrt(mean_squared_error(y, blended_oof)))
    row = {f'weight_{name}': float(w[i]) for i, name in enumerate(models)}
    row['rmse'] = rmse
    blend_rows.append(row)

blend_df = pd.DataFrame(blend_rows).sort_values('rmse', ascending=True)
blend_path = RESULTS_DIR / 'stack_constrained_weights.csv'
blend_df.to_csv(blend_path, index=False)
print('Saved:', blend_path.resolve())

best_row = blend_df.iloc[0]
best_rmse = float(best_row['rmse'])
best_weights = np.array([best_row[f'weight_{name}'] for name in models], dtype=float)

stacked_test = X_meta_test @ best_weights

coeffs_df = pd.DataFrame({'model': models, 'coef': best_weights})
coeffs_path = RESULTS_DIR / 'stack_constrained_coeffs.csv'
coeffs_df.to_csv(coeffs_path, index=False)
print('Saved:', coeffs_path.resolve())

submission = sub_df.copy()
submission['BeatsPerMinute'] = stacked_test
submission_path = RESULTS_DIR / 'submission_stacking_constrained.csv'
submission.to_csv(submission_path, index=False)
print('Saved:', submission_path.resolve())
print('\nSubmission head:')
print(submission.head())

print('\nStacking summary:')
print('Models:', models)
print('Best constrained RMSE:', best_rmse)
print('Weights:', best_weights.tolist())
